In [1]:
import pandas as pd

data = pd.read_csv('fraudtrain.csv')


c:\Users\smadh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.sparse import csr_matrix, vstack


def process_data_chunks(file_path, chunk_size):
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    sparse_chunks = []
    
   
    scaler = StandardScaler()
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=True)
    
    for chunk in chunks:
       
        chunk['trans_hour'] = pd.to_datetime(chunk['trans_date_trans_time']).dt.hour
        chunk['trans_day_of_week'] = pd.to_datetime(chunk['trans_date_trans_time']).dt.dayofweek
        
        
        numerical_features = ['amt', 'lat', 'long', 'city_pop']
        chunk[numerical_features] = scaler.fit_transform(chunk[numerical_features])
        

        categorical_features = ['merchant', 'category', 'gender']
        encoded_features = encoder.fit_transform(chunk[categorical_features])
        

        sparse_data = csr_matrix(encoded_features)
        sparse_chunks.append(sparse_data)
    
  
    final_sparse_matrix = vstack(sparse_chunks)
    
    return final_sparse_matrix

# Example usage
file_path = 'fraudtrain.csv'  
chunk_size = 100000  

final_sparse_matrix = process_data_chunks(file_path, chunk_size)
print("Final sparse matrix shape:", final_sparse_matrix.shape)


c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

Final sparse matrix shape: (1296675, 709)


c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Isolation Forest - Anamoly Method

In [6]:

from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score


X = final_sparse_matrix
y = data['is_fraud']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score

# Example of hyperparameter tuning
best_score = 0
best_model = None
for contamination in [0.01, 0.02, 0.05]:
    for n_estimators in [100, 200, 300]:
        iso_forest = IsolationForest(contamination=contamination, n_estimators=n_estimators, random_state=42)
        iso_forest.fit(X_train)
        y_pred_iso = iso_forest.predict(X_test)
        roc_auc = roc_auc_score(y_test, y_pred_iso)
        print(f'Contamination={contamination}, n_estimators={n_estimators}, ROC AUC Score: {roc_auc}')
        if roc_auc > best_score:
            best_score = roc_auc
            best_model = iso_forest

print(f'Best ROC AUC Score: {best_score}')


Contamination=0.01, n_estimators=100, ROC AUC Score: 0.5010884423959653
Contamination=0.01, n_estimators=200, ROC AUC Score: 0.501038765046744
Contamination=0.01, n_estimators=300, ROC AUC Score: 0.5016353461835649
Contamination=0.02, n_estimators=100, ROC AUC Score: 0.49921203315923185
Contamination=0.02, n_estimators=200, ROC AUC Score: 0.5010771123622916
Contamination=0.02, n_estimators=300, ROC AUC Score: 0.5031807347067512
Contamination=0.05, n_estimators=100, ROC AUC Score: 0.5009252669447798
Contamination=0.05, n_estimators=200, ROC AUC Score: 0.5078048174588674
Contamination=0.05, n_estimators=300, ROC AUC Score: 0.5060747481108955
Best ROC AUC Score: 0.5078048174588674


OverSampling the Data using Smote

In [7]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=42)

rf_clf.fit(X_train_smote, y_train_smote)

y_pred_rf = rf_clf.predict(X_test)


print(classification_report(y_test, y_pred_rf))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_rf)}')


Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

logreg_clf = LogisticRegression(random_state=42)

logreg_clf.fit(X_train_smote, y_train_smote)


y_pred_logreg = logreg_clf.predict(X_test)


print(classification_report(y_test, y_pred_logreg))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_logreg)}')


c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       1.00      0.65      0.79    257815
           1       0.01      0.70      0.02      1520

    accuracy                           0.65    259335
   macro avg       0.50      0.67      0.40    259335
weighted avg       0.99      0.65      0.78    259335

ROC AUC Score: 0.6745478640334716


Random UnderSampling

In [11]:
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score


print("Original training dataset shape:", Counter(y_train))


rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

print("Resampled training dataset shape:", Counter(y_train_rus))

Original training dataset shape: Counter({0: 1031354, 1: 5986})
Resampled training dataset shape: Counter({0: 5986, 1: 5986})


LogisticRegression

In [12]:
from sklearn.linear_model import LogisticRegression

logreg_clf = LogisticRegression(random_state=42)


logreg_clf.fit(X_train_rus, y_train_rus)


y_pred_logreg = logreg_clf.predict(X_test)


print(classification_report(y_test, y_pred_logreg))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_logreg)}')

c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       1.00      0.69      0.81    257815
           1       0.01      0.67      0.02      1520

    accuracy                           0.69    259335
   macro avg       0.50      0.68      0.42    259335
weighted avg       0.99      0.69      0.81    259335

ROC AUC Score: 0.6788795540866206


Cluster Centroids Undersampling:

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import ClusterCentroids
from sklearn.metrics import classification_report, roc_auc_score


print("Original training dataset shape:", Counter(y_train))

cc = ClusterCentroids(random_state=42)
X_train_cc, y_train_cc = cc.fit_resample(X_train, y_train)

print("Resampled training dataset shape:", Counter(y_train_cc))



In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_clf = LogisticRegression(random_state=42)

logreg_clf.fit(X_train_cc, y_train_cc)

y_pred_logreg = logreg_clf.predict(X_test)

print(classification_report(y_test, y_pred_logreg))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_logreg)}')

Class Weight Adjustment

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

print("Original training dataset shape:", Counter(y_train))


model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print(classification_report(y_test, y_pred))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred)}')


Original training dataset shape: Counter({0: 1031354, 1: 1031354})


c:\Users\smadh\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       1.00      0.65      0.79    257815
           1       0.01      0.70      0.02      1520

    accuracy                           0.65    259335
   macro avg       0.50      0.67      0.40    259335
weighted avg       0.99      0.65      0.78    259335

ROC AUC Score: 0.6745478640334716


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score


print("Training set shape after SMOTE:", X_train_smote.shape)
print("Testing set shape:", X_test.shape)


svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_smote, y_train_smote)


y_pred_svm = svm_model.predict(X_test)

print("SVM Model:")
print(classification_report(y_test, y_pred_svm))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_svm)}')


Training set shape after SMOTE: (2062708, 709)
Testing set shape: (259335, 709)


Ensemble Methods

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)

brf_model.fit(X_train, y_train)


y_pred_brf = brf_model.predict(X_test)


print("Balanced Random Forest Model:")
print(classification_report(y_test, y_pred_brf))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred_brf)}')
